## Python2Vec

In [44]:
import pandas as pd
import numpy as np
import ast
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

In [2]:
train_data = open('python.txt', 'r')
sentence = LineSentence(train_data)

model = Word2Vec()

model.build_vocab(sentence)
model.train(sentence, total_examples=model.corpus_count, epochs=model.epochs)

(20533662, 26441915)

In [43]:
id_freq_df = pd.read_csv('identifier_frequency.csv', index_col=0)
top_ids = id_freq_df['identifier'][:50].to_numpy()

compare_top_ids = np.zeros((50,50))

for i in range(50):
    for j in range(50):

        if i==j:
            compare_top_ids[i][j] = 0
        
        else:
            compare_top_ids[i][j] = model.wv.similarity(top_ids[i], top_ids[j])

most_similar = np.argmax(compare_top_ids, axis=0)

similarity = []

for i in range(50):
    similarity.append(model.wv.similarity(top_ids[i], top_ids[most_similar[i]]))

similarity_df = pd.DataFrame()

similarity_df['word'] = top_ids
similarity_df['most_similar'] = top_ids[most_similar]
similarity_df['similarity'] = similarity

similarity_df.sort_values('similarity', ascending=False, inplace=True)
similarity_df.reset_index(inplace=True)
similarity_df.drop('index', axis=1, inplace=True)
similarity_df.to_csv('most_similar_identifiers.csv')

In [57]:
example_reader = open('example.py', 'r')

code = example_reader.read()

ast_tree = ast.parse(code)

for node in ast.walk(ast_tree):
    if isinstance(node, ast.FunctionDef):
        snippet = []
        #print(ast.dump(node, '\n\n\n'))
        for n in ast.iter_child_nodes(node):
            snippet.append(n.)

        print(snippet)

['<ast.arguments object at 0x7fb819d4e7a0>', '<ast.Expr object at 0x7fb819d4e710>', '<ast.Import object at 0x7fb819d4e6b0>', '<ast.With object at 0x7fb819d4e650>', '<ast.Return object at 0x7fb81a7832e0>']
['<ast.arguments object at 0x7fb819d39810>', '<ast.Assign object at 0x7fb819d39a50>', '<ast.Assign object at 0x7fb819d39a80>', '<ast.Assign object at 0x7fb819d384c0>', '<ast.Expr object at 0x7fb819d3b130>']
['<ast.arguments object at 0x7fb81a783a30>', '<ast.Assign object at 0x7fb819d4e020>', '<ast.If object at 0x7fb819d4df00>', '<ast.Assign object at 0x7fb819d4dd20>', '<ast.Expr object at 0x7fb819d4d9c0>']
['<ast.arguments object at 0x7fb819d4d8a0>', '<ast.Expr object at 0x7fb819d4d840>', '<ast.Assign object at 0x7fb819d4d7e0>', '<ast.If object at 0x7fb819d4d6c0>', '<ast.Assign object at 0x7fb819d4d570>', '<ast.FunctionDef object at 0x7fb819d4d390>', '<ast.Assign object at 0x7fb81a03a620>', '<ast.Assign object at 0x7fb81a03ac20>', '<ast.Assign object at 0x7fb81a03a950>', '<ast.Assign 